# Physics-Informed Neural Networks for Reservoir Modeling
## Tutorial 1: Introduction and Motivation

Welcome to this comprehensive tutorial on Physics-Informed Neural Networks (PINNs) for multiphase flow modeling in petroleum reservoirs. This tutorial series will guide you through the complete process of implementing PINNs using real Kansas Geological Survey (KGS) well log data.

### Learning Objectives
By the end of this tutorial, you will understand:
- Why traditional numerical methods have limitations in reservoir modeling
- How pure machine learning approaches fail in physics-constrained problems
- The fundamental concepts behind Physics-Informed Neural Networks
- Essential physics background for reservoir modeling
- Neural network fundamentals for PINN implementation

## 1. The Challenge: Why Traditional Methods Fall Short

### Classical Numerical Methods Limitations

Traditional reservoir simulation relies on finite difference or finite element methods to solve partial differential equations (PDEs) governing fluid flow. However, these approaches face several challenges:

1. **Computational Complexity**: Large-scale reservoir models require enormous computational resources
2. **Grid Dependency**: Solution accuracy depends heavily on mesh quality and resolution
3. **Numerical Dispersion**: Discretization errors can accumulate over time
4. **Boundary Condition Sensitivity**: Complex geometries require sophisticated meshing
5. **Parameter Uncertainty**: Heterogeneous rock properties are difficult to characterize

### Pure Machine Learning Limitations

While ML models can learn patterns from data, they struggle with physics-constrained problems:

1. **Data Scarcity**: Limited well data compared to model complexity
2. **Extrapolation Issues**: Poor performance outside training data range
3. **Physics Violations**: No guarantee that predictions satisfy physical laws
4. **Interpretability**: Black-box nature makes results difficult to validate
5. **Conservation Laws**: Cannot enforce mass, momentum, or energy conservation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from pathlib import Path

# Set up plotting style
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

print("Environment setup complete!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

## 2. The Solution: Physics-Informed Neural Networks

PINNs represent a paradigm shift that combines the flexibility of neural networks with the rigor of physics-based modeling. The key insight is to embed physical laws directly into the neural network training process.

### Core PINN Concept

Instead of just fitting data, PINNs minimize a composite loss function:

$$\mathcal{L}_{total} = \mathcal{L}_{data} + \lambda_{PDE} \mathcal{L}_{PDE} + \lambda_{BC} \mathcal{L}_{BC}$$

Where:
- $\mathcal{L}_{data}$: Traditional data fitting loss
- $\mathcal{L}_{PDE}$: Physics equation residuals
- $\mathcal{L}_{BC}$: Boundary condition violations
- $\lambda$: Weighting parameters

### Advantages of PINNs

1. **Physics Consistency**: Solutions automatically satisfy governing equations
2. **Data Efficiency**: Can work with sparse, noisy data
3. **Mesh-Free**: No need for complex grid generation
4. **Uncertainty Quantification**: Natural framework for Bayesian inference
5. **Inverse Problems**: Can simultaneously learn parameters and solutions

In [ ]:
# Demonstrate the concept with a simple 1D example
def visualize_pinn_concept():
    """Visualize how PINNs combine data and physics constraints"""
    
    # Create synthetic data
    x = np.linspace(0, 1, 100)
    
    # True solution (for demonstration)
    y_true = np.sin(np.pi * x)
    
    # Sparse noisy observations
    x_obs = np.array([0.2, 0.4, 0.6, 0.8])
    y_obs = np.sin(np.pi * x_obs) + 0.1 * np.random.randn(len(x_obs))
    
    # Pure ML fit (polynomial)
    coeffs = np.polyfit(x_obs, y_obs, 3)
    y_ml = np.polyval(coeffs, x)
    
    # Physics-informed solution (satisfies d²y/dx² + π²y = 0)
    y_pinn = np.sin(np.pi * x)  # Simplified for illustration
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Plot 1: Data fitting comparison
    ax1.plot(x, y_true, 'k-', linewidth=2, label='True Solution', alpha=0.7)
    ax1.plot(x, y_ml, 'r--', linewidth=2, label='Pure ML (Polynomial Fit)')
    ax1.plot(x, y_pinn, 'b-', linewidth=2, label='PINN Solution')
    ax1.scatter(x_obs, y_obs, color='red', s=100, zorder=5, label='Observations')
    ax1.set_xlabel('Position (x)')
    ax1.set_ylabel('Solution (y)')
    ax1.set_title('Data Fitting: ML vs PINN')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Plot 2: Physics residual
    # For the PDE: d²y/dx² + π²y = 0
    residual_ml = np.gradient(np.gradient(y_ml, x), x) + np.pi**2 * y_ml
    residual_pinn = np.gradient(np.gradient(y_pinn, x), x) + np.pi**2 * y_pinn
    
    ax2.plot(x, np.abs(residual_ml), 'r--', linewidth=2, label='ML Residual')
    ax2.plot(x, np.abs(residual_pinn), 'b-', linewidth=2, label='PINN Residual')
    ax2.set_xlabel('Position (x)')
    ax2.set_ylabel('|PDE Residual|')
    ax2.set_title('Physics Constraint Satisfaction')
    ax2.set_yscale('log')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("Key Insight: PINNs enforce physics even with sparse data!")
    print(f"ML PDE residual (max): {np.max(np.abs(residual_ml)):.3f}")
    print(f"PINN PDE residual (max): {np.max(np.abs(residual_pinn)):.6f}")

visualize_pinn_concept()

## 3. Essential Physics Background

To understand PINNs for reservoir modeling, we need to review the fundamental physics governing fluid flow in porous media.

### 3.1 Darcy's Law

Darcy's law describes fluid flow through porous media:

$$\vec{v} = -\frac{k}{\mu} \nabla p$$

Where:
- $\vec{v}$: Darcy velocity (m/s)
- $k$: Permeability (m²)
- $\mu$: Fluid viscosity (Pa·s)
- $p$: Pressure (Pa)

### 3.2 Continuity Equation

Mass conservation for incompressible flow:

$$\nabla \cdot \vec{v} = 0$$

Combining with Darcy's law:

$$\nabla \cdot \left(\frac{k}{\mu} \nabla p\right) = 0$$

### 3.3 Buckley-Leverett Equation

For two-phase flow (oil-water), the saturation evolution follows:

$$\frac{\partial S_w}{\partial t} + \frac{\partial f_w}{\partial x} = 0$$

Where:
- $S_w$: Water saturation
- $f_w$: Fractional flow function
- $t$: Time
- $x$: Spatial coordinate

In [ ]:
# Visualize key reservoir properties
def visualize_reservoir_properties():
    """Demonstrate relationships between key reservoir properties"""
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))
    
    # 1. Porosity distribution
    porosity = np.random.lognormal(mean=-1.5, sigma=0.3, size=1000)
    porosity = np.clip(porosity, 0.05, 0.35)
    
    ax1.hist(porosity, bins=30, alpha=0.7, color='skyblue', edgecolor='black')
    ax1.set_xlabel('Porosity (fraction)')
    ax1.set_ylabel('Frequency')
    ax1.set_title('Typical Porosity Distribution')
    ax1.axvline(np.mean(porosity), color='red', linestyle='--', 
                label=f'Mean: {np.mean(porosity):.3f}')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 2. Permeability-Porosity relationship
    # Kozeny-Carman relationship: k ∝ φ³/(1-φ)²
    phi = np.linspace(0.05, 0.35, 100)
    k_kozeny = 1e-12 * (phi**3) / ((1-phi)**2)  # m²
    k_kozeny_md = k_kozeny * 1e15  # Convert to millidarcies
    
    ax2.loglog(phi, k_kozeny_md, 'b-', linewidth=2, label='Kozeny-Carman')
    ax2.scatter(porosity[:100], 10**(3*np.log10(porosity[:100]) + np.random.normal(0, 0.5, 100)), 
                alpha=0.6, color='red', s=20, label='Typical Data')
    ax2.set_xlabel('Porosity (fraction)')
    ax2.set_ylabel('Permeability (mD)')
    ax2.set_title('Porosity-Permeability Relationship')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # 3. Fractional flow curve
    Sw = np.linspace(0.2, 0.8, 100)
    # Simplified fractional flow (Corey model)
    krw = ((Sw - 0.2) / 0.6) ** 2
    kro = ((0.8 - Sw) / 0.6) ** 2
    mu_w, mu_o = 1e-3, 5e-3  # Pa·s
    fw = krw / mu_w / (krw / mu_w + kro / mu_o)
    
    ax3.plot(Sw, fw, 'g-', linewidth=3, label='Fractional Flow')
    ax3.plot(Sw, Sw, 'k--', alpha=0.5, label='45° Line')
    ax3.set_xlabel('Water Saturation')
    ax3.set_ylabel('Water Fractional Flow')
    ax3.set_title('Buckley-Leverett Fractional Flow')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # 4. Pressure field example
    x = np.linspace(0, 100, 50)
    y = np.linspace(0, 100, 50)
    X, Y = np.meshgrid(x, y)
    
    # Simple pressure field (injection-production)
    P = 100 - 0.5 * np.sqrt((X-20)**2 + (Y-50)**2) + 0.3 * np.sqrt((X-80)**2 + (Y-50)**2)
    
    contour = ax4.contourf(X, Y, P, levels=20, cmap='viridis')
    ax4.contour(X, Y, P, levels=10, colors='white', alpha=0.6, linewidths=0.5)
    plt.colorbar(contour, ax=ax4, label='Pressure (bar)')
    ax4.plot(20, 50, 'ro', markersize=10, label='Injector')
    ax4.plot(80, 50, 'bs', markersize=10, label='Producer')
    ax4.set_xlabel('X (m)')
    ax4.set_ylabel('Y (m)')
    ax4.set_title('Pressure Field Example')
    ax4.legend()
    
    plt.tight_layout()
    plt.show()
    
    print("Physics Insights:")
    print(f"• Porosity typically ranges from 5-35% in reservoirs")
    print(f"• Permeability varies over many orders of magnitude")
    print(f"• Fractional flow determines saturation evolution")
    print(f"• Pressure drives fluid flow according to Darcy's law")

visualize_reservoir_properties()

## 4. Neural Network Fundamentals for PINNs

Understanding neural networks is crucial for implementing PINNs effectively. Let's review the key concepts.

### 4.1 Universal Approximation

Neural networks can approximate any continuous function given sufficient width and depth. For PINNs, this means we can represent complex pressure and saturation fields.

### 4.2 Automatic Differentiation

The key to PINNs is computing derivatives of neural network outputs with respect to inputs. This enables us to evaluate PDE residuals.

### 4.3 Activation Functions

For PINNs, smooth activation functions are preferred:
- **Tanh**: Smooth, bounded, good for physics problems
- **Swish**: $f(x) = x \cdot \sigma(x)$, smooth everywhere
- **Sine**: Periodic, useful for wave-like solutions

In [ ]:
# Demonstrate automatic differentiation for PINNs
class SimplePINN(nn.Module):
    """Simple PINN for demonstration"""
    
    def __init__(self, input_dim=2, hidden_dim=50, output_dim=1):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, output_dim)
        )
    
    def forward(self, x):
        return self.layers(x)

def demonstrate_autodiff():
    """Show how automatic differentiation works in PINNs"""
    
    # Create a simple PINN
    model = SimplePINN(input_dim=2, hidden_dim=20, output_dim=1)
    
    # Create input points (x, y coordinates)
    x = torch.linspace(0, 1, 50, requires_grad=True).reshape(-1, 1)
    y = torch.linspace(0, 1, 50, requires_grad=True).reshape(-1, 1)
    
    # Create grid
    X, Y = torch.meshgrid(x.squeeze(), y.squeeze(), indexing='ij')
    coords = torch.stack([X.flatten(), Y.flatten()], dim=1)
    coords.requires_grad_(True)
    
    # Forward pass
    u = model(coords)
    
    # Compute gradients (this is the key for PINNs!)
    u_x = torch.autograd.grad(u.sum(), coords, create_graph=True)[0][:, 0]
    u_y = torch.autograd.grad(u.sum(), coords, create_graph=True)[0][:, 1]
    
    # Second derivatives
    u_xx = torch.autograd.grad(u_x.sum(), coords, create_graph=True)[0][:, 0]
    u_yy = torch.autograd.grad(u_y.sum(), coords, create_graph=True)[0][:, 1]
    
    # Laplacian (for Laplace equation: ∇²u = 0)
    laplacian = u_xx + u_yy
    
    # Visualize
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))
    
    # Solution field
    u_grid = u.detach().numpy().reshape(50, 50)
    im1 = ax1.imshow(u_grid, extent=[0, 1, 0, 1], origin='lower', cmap='viridis')
    ax1.set_title('Neural Network Output u(x,y)')
    ax1.set_xlabel('x')
    ax1.set_ylabel('y')
    plt.colorbar(im1, ax=ax1)
    
    # X-derivative
    ux_grid = u_x.detach().numpy().reshape(50, 50)
    im2 = ax2.imshow(ux_grid, extent=[0, 1, 0, 1], origin='lower', cmap='RdBu')
    ax2.set_title('∂u/∂x')
    ax2.set_xlabel('x')
    ax2.set_ylabel('y')
    plt.colorbar(im2, ax=ax2)
    
    # Y-derivative
    uy_grid = u_y.detach().numpy().reshape(50, 50)
    im3 = ax3.imshow(uy_grid, extent=[0, 1, 0, 1], origin='lower', cmap='RdBu')
    ax3.set_title('∂u/∂y')
    ax3.set_xlabel('x')
    ax3.set_ylabel('y')
    plt.colorbar(im3, ax=ax3)
    
    # Laplacian (PDE residual)
    lap_grid = laplacian.detach().numpy().reshape(50, 50)
    im4 = ax4.imshow(lap_grid, extent=[0, 1, 0, 1], origin='lower', cmap='seismic')
    ax4.set_title('∇²u (PDE Residual)')
    ax4.set_xlabel('x')
    ax4.set_ylabel('y')
    plt.colorbar(im4, ax=ax4)
    
    plt.tight_layout()
    plt.show()
    
    print("Automatic Differentiation Demo:")
    print(f"• Neural network output shape: {u.shape}")
    print(f"• First derivatives computed automatically")
    print(f"• Second derivatives for PDE residuals")
    print(f"• Mean Laplacian residual: {torch.mean(torch.abs(laplacian)):.6f}")
    print("\nThis is how PINNs enforce physics constraints!")

demonstrate_autodiff()

## 5. Well Log Data and Reservoir Characterization

Understanding well log data is essential for our PINN implementation. Well logs provide crucial information about reservoir properties.

### 5.1 Key Well Log Curves

- **Gamma Ray (GR)**: Measures natural radioactivity, indicates clay content
- **Density (RHOB)**: Bulk density, related to porosity
- **Neutron Porosity (NPHI)**: Hydrogen content, indicates porosity
- **Resistivity (RT)**: Electrical resistance, indicates fluid saturation
- **Photoelectric Factor (PEF)**: Lithology indicator

### 5.2 LAS File Format

Log ASCII Standard (LAS) files contain:
- Header information (well location, dates, units)
- Curve definitions and units
- Depth-indexed measurement data

In [ ]:
# Simulate typical well log data
def simulate_well_log_data():
    """Create synthetic well log data for demonstration"""
    
    # Depth array (feet)
    depth = np.linspace(2000, 2500, 500)
    
    # Simulate realistic well log curves
    np.random.seed(42)
    
    # Gamma Ray (API units) - higher in shales
    gr_base = 30 + 50 * np.sin(0.01 * depth) + 10 * np.random.randn(len(depth))
    gr = np.clip(gr_base, 10, 150)
    
    # Porosity (fraction) - inversely related to GR
    phi_base = 0.25 - 0.001 * (gr - 50) + 0.02 * np.random.randn(len(depth))
    phi = np.clip(phi_base, 0.05, 0.35)
    
    # Permeability (mD) - related to porosity
    k_base = 100 * (phi / 0.2) ** 3 * np.exp(np.random.randn(len(depth)))
    k = np.clip(k_base, 0.1, 1000)
    
    # Resistivity (ohm-m) - varies with saturation
    rt_base = 10 + 20 * (1 - phi) + 5 * np.random.randn(len(depth))
    rt = np.clip(rt_base, 1, 100)
    
    # Water saturation (Archie's equation approximation)
    sw = np.clip(0.3 + 0.4 * (10 / rt) ** 0.5, 0.2, 1.0)
    
    return {
        'DEPT': depth,
        'GR': gr,
        'PHIE': phi,
        'PERM': k,
        'RT': rt,
        'SW': sw
    }

def plot_well_log_data():
    """Create a typical well log display"""
    
    data = simulate_well_log_data()
    depth = data['DEPT']
    
    fig, axes = plt.subplots(1, 5, figsize=(20, 10), sharey=True)
    
    # Track 1: Gamma Ray
    axes[0].plot(data['GR'], depth, 'g-', linewidth=1)
    axes[0].fill_betweenx(depth, 0, data['GR'], alpha=0.3, color='green')
    axes[0].set_xlabel('Gamma Ray\n(API)')
    axes[0].set_xlim(0, 150)
    axes[0].grid(True, alpha=0.3)
    axes[0].invert_yaxis()
    
    # Track 2: Porosity
    axes[1].plot(data['PHIE'], depth, 'b-', linewidth=1, label='Porosity')
    axes[1].set_xlabel('Porosity\n(fraction)')
    axes[1].set_xlim(0, 0.4)
    axes[1].grid(True, alpha=0.3)
    
    # Track 3: Permeability (log scale)
    axes[2].semilogx(data['PERM'], depth, 'r-', linewidth=1)
    axes[2].set_xlabel('Permeability\n(mD)')
    axes[2].set_xlim(0.1, 1000)
    axes[2].grid(True, alpha=0.3)
    
    # Track 4: Resistivity (log scale)
    axes[3].semilogx(data['RT'], depth, 'm-', linewidth=1)
    axes[3].set_xlabel('Resistivity\n(ohm-m)')
    axes[3].set_xlim(1, 100)
    axes[3].grid(True, alpha=0.3)
    
    # Track 5: Water Saturation
    axes[4].plot(data['SW'], depth, 'c-', linewidth=1)
    axes[4].fill_betweenx(depth, 0, data['SW'], alpha=0.3, color='cyan', label='Water')
    axes[4].fill_betweenx(depth, data['SW'], 1, alpha=0.3, color='brown', label='Oil')
    axes[4].set_xlabel('Water Saturation\n(fraction)')
    axes[4].set_xlim(0, 1)
    axes[4].grid(True, alpha=0.3)
    axes[4].legend()
    
    # Set common y-axis
    axes[0].set_ylabel('Depth (ft)')
    
    plt.suptitle('Typical Well Log Display', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # Statistics
    print("Well Log Statistics:")
    for key, values in data.items():
        if key != 'DEPT':
            print(f"{key:4s}: mean={np.mean(values):6.3f}, std={np.std(values):6.3f}, "
                  f"min={np.min(values):6.3f}, max={np.max(values):6.3f}")

plot_well_log_data()

## 6. Summary and Next Steps

In this introduction, we've covered:

### Key Concepts Learned
1. **Limitations** of traditional numerical methods and pure ML approaches
2. **PINN Philosophy**: Embedding physics constraints in neural network training
3. **Essential Physics**: Darcy's law, continuity equation, Buckley-Leverett equation
4. **Neural Network Fundamentals**: Automatic differentiation and activation functions
5. **Well Log Data**: Understanding reservoir characterization from measurements

### Why PINNs Matter for Reservoir Modeling
- **Physics Consistency**: Solutions automatically satisfy governing equations
- **Data Efficiency**: Work with sparse, noisy well data
- **Flexibility**: Handle complex geometries without meshing
- **Uncertainty**: Natural framework for probabilistic modeling

### Coming Up Next
In the following tutorials, we will:
1. **Tutorial 2**: Process real KGS LAS files and build datasets
2. **Tutorial 3**: Implement PINN architecture with physics constraints
3. **Tutorial 4**: Train PINNs with optimization best practices
4. **Tutorial 5**: Validate results and analyze performance
5. **Tutorial 6**: Explore advanced topics and extensions

### Prerequisites Check
Before proceeding, ensure you understand:
- ✅ Basic calculus and partial derivatives
- ✅ Linear algebra and matrix operations
- ✅ Python programming and NumPy
- ✅ PyTorch basics (tensors, autograd, nn.Module)
- ✅ Basic fluid mechanics concepts

In [ ]:
# Quick knowledge check
def knowledge_check():
    """Interactive knowledge check for key concepts"""
    
    questions = [
        {
            'question': 'What is the main advantage of PINNs over pure ML approaches?',
            'options': ['Faster training', 'Physics constraints', 'Better accuracy', 'Simpler implementation'],
            'answer': 1
        },
        {
            'question': 'Which equation describes fluid flow in porous media?',
            'options': ['Navier-Stokes', 'Darcy\'s Law', 'Bernoulli', 'Euler'],
            'answer': 1
        },
        {
            'question': 'What enables PINNs to compute PDE residuals?',
            'options': ['Finite differences', 'Automatic differentiation', 'Numerical integration', 'Interpolation'],
            'answer': 1
        }
    ]
    
    print("🧠 Knowledge Check - Answer the questions below:")
    print("=" * 50)
    
    for i, q in enumerate(questions, 1):
        print(f"\nQuestion {i}: {q['question']}")
        for j, option in enumerate(q['options']):
            print(f"  {j+1}. {option}")
        
        # In a real interactive notebook, you'd get user input
        # For demonstration, we'll show the correct answer
        correct = q['options'][q['answer']]
        print(f"  ✅ Correct answer: {correct}")
    
    print("\n🎉 Great! You're ready for the next tutorial.")
    print("\nNext: Tutorial 2 - Data Processing and LAS File Handling")

knowledge_check()